In [52]:
import pandas as pd

In [36]:
# Database Details
dbDetails = {
    "driver": "postgresql",
    "username": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432",
    "database": "alpha_emami"
}
dbDetails["uri"] = '%(driver)s://%(username)s:%(password)s@%(host)s:%(port)s/%(database)s' % dbDetails


In [37]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [38]:
postgres_engine = create_engine(dbDetails["uri"], echo=False)
Sessions= sessionmaker(bind=postgres_engine)
sesion = Sessions()

In [39]:
#SQL Alchemy Model
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Date, Numeric

Base = declarative_base()

class StockPrimary(Base):
    # table properties
    __tablename__ = 'stock_primary'
    # table columns
    source_tag = Column(Integer, primary_key=True, autoincrement=False)
    status_date = Column(Date, primary_key=True, autoincrement=False)
    sku_code = Column(String(length=256), primary_key=True)
    sc_node_code = Column(String(length=256), primary_key=True)
    sales_channel_code = Column(String(length=256), primary_key=True)
    stock_type = Column(Integer, primary_key=True)
    storage_loc = Column(String(length=256), primary_key=True)
    batch_code = Column(String(length=256), primary_key=True)
    date_of_mfg = Column(Date)
    date_of_expiry = Column(Date)
    uom = Column(String(length=256),primary_key=True,default="NA")
    stock_qty = Column(Numeric(precision=38, scale=4))
    stock_value = Column(Numeric(precision=38, scale=4))

In [41]:
sq = sesion.query(StockPrimary)

In [42]:
#configure cols to group and agg
groupbycols = ['source_tag','status_date','sku_code','sc_node_code','sales_channel_code']
groupby_cols_agg = {"stock_qty" : {'sum' : "total_stocks"}}

In [43]:
groupbycols = [getattr(StockPrimary,col) for col in groupbycols]

In [45]:
sq = sq.group_by(*groupbycols)

In [46]:
sq

In [47]:
from sqlalchemy import func,and_

In [48]:
agg_cols = list()
for col, agg_fn_info in groupby_cols_agg.items():
    print (col,agg_fn_info)
    column_id = getattr(StockPrimary, col)
    for function_name, label in agg_fn_info.items():
        function = getattr(func, function_name)
        agg_cols.append(function(column_id).label(label))
agg_cols

stock_qty {'sum': 'total_stocks'}


In [49]:
sq = sq.with_entities(*groupbycols, *agg_cols)

In [50]:
print (sq)

SELECT stock_primary.source_tag AS stock_primary_source_tag, stock_primary.status_date AS stock_primary_status_date, stock_primary.sku_code AS stock_primary_sku_code, stock_primary.sc_node_code AS stock_primary_sc_node_code, stock_primary.sales_channel_code AS stock_primary_sales_channel_code, sum(stock_primary.stock_qty) AS total_stocks 
FROM stock_primary GROUP BY stock_primary.source_tag, stock_primary.status_date, stock_primary.sku_code, stock_primary.sc_node_code, stock_primary.sales_channel_code, stock_primary.source_tag, stock_primary.status_date, stock_primary.sku_code, stock_primary.sc_node_code, stock_primary.sales_channel_code


In [51]:
df = pd.read_sql(sq.statement, sesion.bind)

In [53]:
df.head()

,source_tag,status_date,sku_code,sc_node_code,sales_channel_code,total_stocks
0,1,2019-04-01,'1000400042801,DP08,ALL,0.0
1,1,2019-04-01,'1000400042801,DP14,ALL,0.0
2,1,2019-04-01,'1000400074201,DP01,ALL,0.0
3,1,2019-04-01,'1000400074201,DP02,ALL,0.0
4,1,2019-04-01,'1000400074201,DP11,ALL,0.0
